In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, median_absolute_error
import math  
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.linear_model import LassoCV

In [2]:
pd.options.display.max_columns = None

data_train = pd.read_csv("train.csv")
display(data_train.head(3))

encounter_id country  patient_id             race  gender      age weight  \
0        533253     USA       70110        Caucasian  Female  [70-80)      ?   
1        426224     USA    29775006  AfricanAmerican    Male  [50-60)      ?   
2        634063     USA    80729253        Caucasian  Female  [60-70)      ?   

  payer_code  outpatient_visits_in_previous_year  \
0          ?                                   0   
1          ?                                   0   
2          ?                                   0   

   emergency_visits_in_previous_year  inpatient_visits_in_previous_year  \
0                                  0                                  2   
1                                  0                                  0   
2                                  0                                  1   

  admission_type       medical_specialty  average_pulse_bpm  \
0      Emergency  Family/GeneralPractice                127   
1      Emergency                       ?                128   
2            NaN  Family/GeneralPractice                 94   

                               discharge_disposition admission_source  \
0                                 Discharged to home   Emergency Room   
1  Discharged/transferred to a federal health car...  Clinic Referral   
2                                 Discharged to home              NaN   

   length_of_stay_in_hospital  number_lab_tests  non_lab_procedures  \
0                           2                48                   0   
1                          14                71                   0   
2                           6                60                   1   

   number_of_medications primary_diagnosis secondary_diagnosis  \
0                     20               515                 276   
1                     25                38                 785   
2                     22               534                 135   

  additional_diagnosis  number_diagnoses glucose_test_result a1c_test_result  \
0                  466                 8                 NaN             NaN   
1                  162                 9                 NaN             NaN   
2                  250                 6                 NaN             NaN   

  change_in_meds_during_hospitalization prescribed_diabetes_meds  \
0                                    No                       No   
1                                    No                      Yes   
2                                    Ch                      Yes   

                   medication readmitted_binary readmitted_multiclass  
0                          []                No              >30 days  
1                 ['insulin']                No                    No  
2  ['glimepiride', 'insulin']                No                    No

In [3]:
data_train = data_train.set_index("encounter_id")

In [4]:
train_data = data_train.copy()

In [5]:
train_data.replace(["", "?", " "], np.nan, inplace = True)
train_data.isna().sum()

country                                      0
patient_id                                   0
race                                      5070
gender                                       0
age                                       3557
weight                                   68990
payer_code                               28201
outpatient_visits_in_previous_year           0
emergency_visits_in_previous_year            0
inpatient_visits_in_previous_year            0
admission_type                            3706
medical_specialty                        34922
average_pulse_bpm                            0
discharge_disposition                     2590
admission_source                          4718
length_of_stay_in_hospital                   0
number_lab_tests                             0
non_lab_procedures                           0
number_of_medications                        0
primary_diagnosis                           16
secondary_diagnosis                        262
additional_di

In [6]:
train_data = train_data.drop(["country", "weight"], axis = 1 )

In [7]:
def process_ch_in_meds(change_in_meds_during_hospitalization):
    if change_in_meds_during_hospitalization == 'Ch':
        return 'Yes'
    else:
        return change_in_meds_during_hospitalization
    
train_data['change_in_meds_during_hospitalization'] = train_data['change_in_meds_during_hospitalization'].apply(process_ch_in_meds)
train_data['change_in_meds_during_hospitalization'] = train_data['change_in_meds_during_hospitalization'].map({'Yes': 1, 'No': 0})

In [8]:
train_data['prescribed_diabetes_meds'] = train_data['prescribed_diabetes_meds'].map({'Yes': 1, 'No': 0})

In [9]:
train_data['readmitted_binary'] = train_data['readmitted_binary'].map({'Yes': 1, 'No': 0})

In [10]:
train_data['age'].value_counts()

[70-80)     17359
[60-70)     14908
[80-90)     11510
[50-60)     11423
[40-50)      6418
[30-40)      2536
[90-100)     1875
[20-30)      1071
[10-20)       474
[0-10)        105
Name: age, dtype: int64

In [11]:
def process_age(age):
    if pd.isnull(age):
        return np.nan
    elif age == '[0-10)':
        return '1'
    elif age == '[10-20)':
        return '2'
    elif age == '[20-30)':
        return '3'
    elif age == '[30-40)':
        return '4'
    elif age == '[40-50)':
        return '5'
    elif age == '[50-60)':
        return '6'
    elif age == '[60-70)':
        return '7'
    elif age == '[70-80)':
        return '8'
    elif age == '[80-90)':
        return '9'
    elif age == '[90-100)':
        return '10'

train_data['age'] = train_data['age'].apply(process_age)

In [12]:
def process_admission_type(admission_type):
    if pd.isnull(admission_type):
        return np.nan
    elif admission_type in {"Not Available", "Not Mapped"}:
        return "Not Available/ Mapped"
    elif admission_type in {"Emergency", "Urgent", "Trauma Center"}:
        return "Emergency"
    elif admission_type == "Newborn":
        return np.nan
    else:
        return admission_type

train_data['admission_type'] = train_data['admission_type'].apply(process_admission_type)


In [13]:
def process_medical_specialty(medical_specialty):
    if medical_specialty in {"InternalMedicine",
    "Family/GeneralPractice",
    "Pediatrics",
    "ObstetricsandGynecology"}:
        return "Primary Care"
    elif medical_specialty in {"Surgery-General",
    "Orthopedics",
    "Orthopedics-Reconstructive"
    "Surgery-Cardiovascular/Thoracic"
    "Surgery-Vascular"
    "Surgery-Neuro"
    "Surgery-Thoracic"
    "Surgery-Cardiovascular"
    "Surgery-Plastic"
    "Surgery-Colon&Rectal"
    "Surgery-Pediatric"
    "Surgery-Maxillofacial"}:
        return "Surgical Specialties"
    elif medical_specialty in {"Cardiology",
    "Nephrology",
    "Radiologist",
    "Pulmonology",
    "Psychiatry",
    "Urology",
    "Gastroenterology"
    "Oncology"
    "Hematology/Oncology",
    "Neurology",
    "Endocrinology",
    "InfectiousDiseases",
    "Rheumatology"
    "AllergyandImmunology",
    "Cardiology-Pediatric",
    "Endocrinology-Metabolism",
    "Dentistry",
    "Neurophysiology",
    "Proctology",
    "Pediatrics-AllergyandImmunology",
    "Pediatrics-Hematology-Oncology",
    "Radiology",
    "PhysicalMedicineandRehabilitation",
    "Otolaryngology",
    "Ophthalmology",
    "Anesthesiology",
    "Pathology"}:
        return "Specialized Medical Fields"
    elif medical_specialty in {"Hospitalist",
    "OutreachServices",
    "PhysicianNotFound",
    "Resident",
    "Speech",
    "SportsMedicine"}:
        return "Other"
    
train_data['medical_specialty'] = train_data['medical_specialty'].apply(process_medical_specialty)

In [14]:
def process_discharge_disposition(discharge_disposition):
    if pd.isnull(discharge_disposition):
        return np.nan
    elif discharge_disposition in {"Expired", "Not Mapped"}:
        return "Not Available/ Mapped"
    elif discharge_disposition in {'Discharged to home',
       'Discharged/transferred to a federal health care facility.',
       'Discharged/transferred to home with home health service',
       'Discharged/transferred to SNF',
       'Discharged/transferred to another short term hospital',
       'Discharged/transferred to ICF',
       'Discharged/transferred to another type of inpatient care institution',
       'Discharged/transferred to another rehab fac including rehab units of a hospital .',
       'Discharged/transferred to a long term care hospital.',
       'Discharged/transferred to home under care of Home IV provider',
       'Discharged/transferred/referred to a psychiatric hospital of psychiatric distinct part unit of a hospital',
       'Discharged/transferred within this institution to Medicare approved swing bed',
       'Discharged/transferred to a nursing facility certified under Medicaid but not certified under Medicare.',
       'Discharged/transferred/referred to this institution for outpatient services',
       'Neonate discharged to another hospital for neonatal aftercare',
       'Discharged/transferred/referred another institution for outpatient services'}:
        return "Discharged"
    elif discharge_disposition in {'Hospice / home', 'Expired at home. Medicaid only, hospice.', 'Hospice / medical facility', "Expired in a medical facility. Medicaid only, hospice." }:
        return "Hospice / Home"
    elif discharge_disposition in {'Left AMA'}:
        return discharge_disposition
    else:
        return np.nan
train_data['discharge_disposition'] = train_data['discharge_disposition'].apply(process_discharge_disposition)

In [15]:
def process_admission_source(admission_source):
    if pd.isnull(admission_source):
        return np.nan
    elif admission_source in {" Not Mapped", " Not Available"}:
        return "Not Available/ Mapped"
    elif admission_source in {" Physician Referral", "Clinic Referral", "HMO Referral" }:
        return "Referral"
    elif admission_source in {"Transfer from a hospital", 
        " Transfer from another health care facility", 
        " Transfer from a Skilled Nursing Facility (SNF)",
        " Transfer from hospital inpt/same fac reslt in a sep claim",      
        " Transfer from critial access hospital",
        " Transfer from Ambulatory Surgery Center"}:
        return "Transfer"
    else:
        return np.nan
    
train_data['admission_source'] = train_data['admission_source'].apply(process_admission_source)

In [16]:
def process_diagnosis(diagnosis):
    if isinstance(diagnosis, float) and not pd.notna(diagnosis):
        return "Diabetes"  # Handle missing values
    elif isinstance(diagnosis, str):
        if diagnosis[0] == "V":
            return "V"
        elif diagnosis[0] == "E":
            return "E"
        elif "." in diagnosis:
            return "B"
    try:
        diagnosis = int(diagnosis)
        if 1 <= diagnosis < 140: 
            return "Infectious and Parasitic"
        elif 140 <= diagnosis < 240: 
            return "Neoplasms"
        elif 240 <= diagnosis < 280 and not (250 == diagnosis):
            return "Endocrine, Nutritional and Metabolic and Immunity"
        elif diagnosis == 250:
            return "Diabetes"
        elif 280 <= diagnosis < 290:
            return "Blood and Blood forming"
        elif 290 <= diagnosis < 320: 
            return "Mental"
        elif 320 <= diagnosis < 390: 
            return "Nervous"
        elif 390 <= diagnosis < 460: 
            return "Circulatory"
        elif 460 <= diagnosis < 520: 
            return "Respiratory"
        elif 520 <= diagnosis < 580:
            return "Digestive"
        elif 580 <= diagnosis < 630:
            return "Genitourinary"
        elif 630 <= diagnosis < 680: 
            return "Pregnancy Childbirth and Puerperium"
        elif 680 <= diagnosis < 710: 
            return "Skin"
        elif 710 <= diagnosis < 740: 
            return "Musculoskeletal and Connective Tissue"
        elif 740 <= diagnosis < 760: 
            return "Congenital"
        elif 760 <= diagnosis < 780: 
            return "Perinatal Period"
        elif 780 <= diagnosis < 800: 
            return "Symptoms Signs and Ill"
        elif 800 <= diagnosis < 1000:
            return "Injury and Poisoning"
        else:
            return diagnosis
    except ValueError:
        return diagnosis

train_data['primary_diagnosis'] = pd.to_numeric(train_data['primary_diagnosis'], errors='coerce')
train_data['secondary_diagnosis'] = pd.to_numeric(train_data['secondary_diagnosis'], errors='coerce')
train_data['additional_diagnosis'] = pd.to_numeric(train_data['additional_diagnosis'], errors='coerce')

train_data['primary_diagnosis'] = train_data['primary_diagnosis'].apply(process_diagnosis)
train_data['secondary_diagnosis'] = train_data['secondary_diagnosis'].apply(process_diagnosis)
train_data['additional_diagnosis'] = train_data['additional_diagnosis'].apply(process_diagnosis)

In [17]:
train_data['a1c_test_result'].value_counts()

>8      5705
Norm    3503
>7      2708
Name: a1c_test_result, dtype: int64

In [18]:
def process_a1c_test(a1c_test_result):
    if pd.isnull(a1c_test_result):
        return np.nan
    elif a1c_test_result in {">8", ">7"}:
        return "High"
    else:
        return a1c_test_result
train_data['a1c_test_result'] = train_data['a1c_test_result'].apply(process_a1c_test)

In [19]:
train_data['glucose_test_result'].value_counts()

Norm    1806
>200    1055
>300     827
Name: glucose_test_result, dtype: int64

In [20]:
def process_glucose_test(glucose_test_result):
    if pd.isnull(glucose_test_result):
        return np.nan
    elif glucose_test_result in {">200", ">300"}:
        return "High"
    else:
        return glucose_test_result
train_data['glucose_test_result'] = train_data['glucose_test_result'].apply(process_glucose_test)

In [21]:
import re
unique_medications=set()
for meds in train_data["medication"]:
    for compound_med in (re.search(r'(?<=\[)(.*?)(?=\])',meds)[0]).split(sep=','):
        if compound_med!="":
            compound_med=compound_med.strip()
            for med in compound_med.split(sep='-'):
                unique_medications.add(med.strip("'"))
for med in unique_medications:
    train_data[med]=train_data["medication"].map(lambda meds: med in meds).astype("int64")
train_data.drop("medication",axis=1,inplace=True)

In [22]:
# Não corrida 
filling_strategies = {
    'race': 'mode',
    'age': 'mode',
    'payer_code': None,
    'medical_specialty' : 'mode',
    'discharge_disposition' : 'mode',
    'admission_type' : 'mode',
    'admission_source' : 'mode', 
    'glucose_test_result' : None,
    'a1c_test_result' : None,
    'primary_diagnosis' : 'mode',
    'secondary_diagnosis' : 'mode',
    'additional_diagnosis' : 'mode'
}

for column, strategy in filling_strategies.items():
    if strategy == 'mode':
        mode_value = train_data[column].mode().iloc[0]
        train_data[column].fillna(mode_value, inplace=True)
    elif strategy is None:
        train_data[column].fillna('None', inplace=True)
   

In [23]:
train_data['number_of_visits'] = train_data['outpatient_visits_in_previous_year']+ train_data['emergency_visits_in_previous_year'] + train_data['inpatient_visits_in_previous_year']

In [24]:
data = train_data.drop(["readmitted_binary","readmitted_multiclass" ], axis = 1)

target= train_data["readmitted_binary"]

In [25]:
data = pd.get_dummies(data, columns=['gender', 'age','race', 'payer_code', 'admission_type', 'medical_specialty', 'discharge_disposition', 'admission_source', 'primary_diagnosis', 'secondary_diagnosis', 'additional_diagnosis', 'glucose_test_result', 'a1c_test_result', 'change_in_meds_during_hospitalization', 'prescribed_diabetes_meds'])
data

patient_id  outpatient_visits_in_previous_year  \
encounter_id                                                   
533253             70110                                   0   
426224          29775006                                   0   
634063          80729253                                   0   
890610           2919042                                   0   
654194          84871971                                   1   
...                  ...                                 ...   
660160          24531381                                   0   
826429           4663818                                   0   
332030          23397147                                   0   
757560          52161750                                   0   
459757          88410897                                   0   

              emergency_visits_in_previous_year  \
encounter_id                                      
533253                                        0   
426224                                        0   
634063                                        0   
890610                                        0   
654194                                        0   
...                                         ...   
660160                                        1   
826429                                        0   
332030                                        2   
757560                                        0   
459757                                        0   

              inpatient_visits_in_previous_year  average_pulse_bpm  \
encounter_id                                                         
533253                                        2                127   
426224                                        0                128   
634063                                        1                 94   
890610                                        1                 81   
654194                                        0                126   
...                                         ...                ...   
660160                                        0                 90   
826429                                        0                 98   
332030                                        2                137   
757560                                        2                123   
459757                                        1                 62   

              length_of_stay_in_hospital  number_lab_tests  \
encounter_id                                                 
533253                                 2                48   
426224                                14                71   
634063                                 6                60   
890610                                 6                48   
654194                                 6                47   
...                                  ...               ...   
660160                                 7                51   
826429                                 9                20   
332030                                 5                25   
757560                                 2                34   
459757                                 5                42   

              non_lab_procedures  number_of_medications  number_diagnoses  \
encounter_id                                                                
533253                         0                     20                 8   
426224                         0                     25                 9   
634063                         1                     22                 6   
890610                         2                      9                 5   
654194                         1                     15                 9   
...                          ...                    ...               ...   
660160                         1                     18                 9   
826429                         0                      9                 6   
332030                       

In [26]:
data = data.drop('patient_id', axis = 1)

In [27]:
data = data.drop('average_pulse_bpm', axis = 1)

In [28]:
data

outpatient_visits_in_previous_year  \
encounter_id                                       
533253                                         0   
426224                                         0   
634063                                         0   
890610                                         0   
654194                                         1   
...                                          ...   
660160                                         0   
826429                                         0   
332030                                         0   
757560                                         0   
459757                                         0   

              emergency_visits_in_previous_year  \
encounter_id                                      
533253                                        0   
426224                                        0   
634063                                        0   
890610                                        0   
654194                                        0   
...                                         ...   
660160                                        1   
826429                                        0   
332030                                        2   
757560                                        0   
459757                                        0   

              inpatient_visits_in_previous_year  length_of_stay_in_hospital  \
encounter_id                                                                  
533253                                        2                           2   
426224                                        0                          14   
634063                                        1                           6   
890610                                        1                           6   
654194                                        0                           6   
...                                         ...                         ...   
660160                                        0                           7   
826429                                        0                           9   
332030                                        2                           5   
757560                                        2                           2   
459757                                        1                           5   

              number_lab_tests  non_lab_procedures  number_of_medications  \
encounter_id                                                                
533253                      48                   0                     20   
426224                      71                   0                     25   
634063                      60                   1                     22   
890610                      48                   2                      9   
654194                      47                   1                     15   
...                        ...                 ...                    ...   
660160                      51                   1                     18   
826429                      20                   0                      9   
332030                      25                   0                     24   
757560                      34                   1                     13   
459757                      42                   2                      9   

              number_diagnoses  metformin  tolazamide  troglitazone  \
encounter_id                                                          
533253                       8          0           0             0   
426224                       9          0           0             0   
634063                       6          0           0             0   
890610                       5          0           0             0   
654194                       9          0           0             0   
...                        ...        ...         ...           ...   
660160                       9          1           0             0   
8264

In [29]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import time
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import RobustScaler

In [30]:
def avg_score(model):

    skf = StratifiedKFold(n_splits=10)
    score_train = []
    score_test = []
    timer = []
    acc_test = []
    acc_train = []
    
    
    for train_index, test_index in skf.split(data,target):
        X_train, X_val = data.iloc[train_index], data.iloc[test_index]
        y_train, y_val = target.iloc[train_index], target.iloc[test_index]
    
        oversampler = RandomOverSampler(random_state=42)
        X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)


        begin = time.perf_counter()
        model.fit(X_train_resampled, y_train_resampled)
        predict_train = model.predict(X_train_resampled)
        predict_val = model.predict(X_val)
        end = time.perf_counter()
        value_train = f1_score(y_train_resampled, predict_train)
        accuracy_train = model.score(X_train_resampled, y_train_resampled)
        value_test = f1_score(y_val, predict_val)
        accuracy_test = model.score(X_val, y_val)
        score_train.append(value_train)
        score_test.append(value_test)

        acc_train.append(accuracy_train)
        acc_test.append(accuracy_test)

        timer.append(end - begin)

    avg_time = round(np.mean(timer), 3)
    avg_train = round(np.mean(score_train), 3)
    avg_test = round(np.mean(score_test), 3)

    avg_train_a = round(np.mean(acc_train), 3)
    avg_test_a = round(np.mean(acc_test), 3)

    std_time = round(np.std(timer), 2)
    std_train = round(np.std(score_train), 2)
    std_test = round(np.std(score_test), 2)

    std_train_a = round(np.std(acc_train), 2)
    std_test_a = round(np.std(acc_test), 2)

    return (
        str(avg_time) + '+/-' + str(std_time),
        str(avg_train) + '+/-' + str(std_train),
        str(avg_test) + '+/-' + str(std_test),
        str(avg_train_a) + '+/-' + str(std_train_a),
        str(avg_test_a) + '+/-' + str(std_test_a),
    )
    
    return str(avg_time) + '+/-' + str(std_time), str(avg_train) + '+/-' + str(std_train),\
str(avg_test) + '+/-' + str(std_test),str(avg_train_a) + '+/-' + str(std_train_a),\
str(avg_test_a) + '+/-' + str(std_test_a)

def show_results(df, *args):
    """
    Receive an empty dataframe and the different models and call the function avg_score
    """
    count = 0
    
    for arg in args:
        time, avg_train, avg_test ,avg_train_a, avg_test_a= avg_score(arg)
        df.iloc[count] = time, avg_train, avg_test, avg_train_a, avg_test_a
        count+=1
    return df

In [31]:
from sklearn.ensemble import  HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(learning_rate=0.05,
                                       random_state=7,
                                       max_iter= 60,
                                       max_leaf_nodes= 40)

In [31]:
df = pd.DataFrame(columns = ['Time','f1_Train','f1_Test', 'Acc_train', 'Acc_test'], index = [ 'model'])
show_results(df, hgb)

Time     f1_Train       f1_Test    Acc_train      Acc_test
model  5.564+/-0.24  0.659+/-0.0  0.258+/-0.01  0.661+/-0.0  0.644+/-0.01

In [34]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(n_estimators=100, random_state=42)

In [35]:
df = pd.DataFrame(columns = ['Time','f1_Train','f1_Test', 'Acc_train', 'Acc_test'], index = [ 'model'])
show_results(df, gb)

Time     f1_Train       f1_Test    Acc_train      Acc_test
model  45.261+/-3.41  0.601+/-0.0  0.257+/-0.01  0.622+/-0.0  0.655+/-0.01

In [ ]:
gnb =  GaussianNB(var_smoothing=0.0001)
df = pd.DataFrame(columns = ['Time','f1_Train','f1_Test', 'Acc_train', 'Acc_test'], index = [ 'model'])
show_results(df, gnb)

In [36]:
def avg_score2(model, scale):

    skf = StratifiedKFold(n_splits=10)
    score_train = []
    score_test = []
    timer = []
    acc_test = []
    acc_train = []
    
    
    for train_index, test_index in skf.split(data,target):
        X_train, X_val = data.iloc[train_index], data.iloc[test_index]
        y_train, y_val = target.iloc[train_index], target.iloc[test_index]
    
        oversampler = RandomOverSampler(random_state=42)
        X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
        

        if scale == 'minmax':
            
            min_max = MinMaxScaler().fit(X_train_resampled)
            scaled_X_train = min_max.transform(X_train_resampled)
            scaled_X_val = min_max.transform(X_val)
        
        elif scale == 'robust':
            robust = RobustScaler().fit(X_train_resampled)
            scaled_X_train = robust.transform(X_train_resampled)
            scaled_X_val = robust.transform(X_val)

        begin = time.perf_counter()
        model.fit(scaled_X_train, y_train_resampled)
        predict_train = model.predict(scaled_X_train)
        predict_val = model.predict(scaled_X_val)
        end = time.perf_counter()
        value_train = f1_score(y_train_resampled, predict_train)
        accuracy_train = model.score(scaled_X_train, y_train_resampled)
        value_test = f1_score(y_val, predict_val)
        accuracy_test = model.score(scaled_X_val, y_val)
        score_train.append(value_train)
        score_test.append(value_test)

        acc_train.append(accuracy_train)
        acc_test.append(accuracy_test)

        timer.append(end - begin)

    avg_time = round(np.mean(timer), 3)
    avg_train = round(np.mean(score_train), 3)
    avg_test = round(np.mean(score_test), 3)

    avg_train_a = round(np.mean(acc_train), 3)
    avg_test_a = round(np.mean(acc_test), 3)

    std_time = round(np.std(timer), 2)
    std_train = round(np.std(score_train), 2)
    std_test = round(np.std(score_test), 2)

    std_train_a = round(np.std(acc_train), 2)
    std_test_a = round(np.std(acc_test), 2)

    return (
        str(avg_time) + '+/-' + str(std_time),
        str(avg_train) + '+/-' + str(std_train),
        str(avg_test) + '+/-' + str(std_test),
        str(avg_train_a) + '+/-' + str(std_train_a),
        str(avg_test_a) + '+/-' + str(std_test_a),
    )
    
    return str(avg_time) + '+/-' + str(std_time), str(avg_train) + '+/-' + str(std_train),\
str(avg_test) + '+/-' + str(std_test),str(avg_train_a) + '+/-' + str(std_train_a),\
str(avg_test_a) + '+/-' + str(std_test_a)

def show_results2(df, scale, *args):
    """
    Receive an empty dataframe and the different models and call the function avg_score
    """
    count = 0
    
    for arg in args:
        time, avg_train, avg_test ,avg_train_a, avg_test_a= avg_score(arg, scale)
        df.iloc[count] = time, avg_train, avg_test, avg_train_a, avg_test_a
        count+=1
    return df

In [41]:
from sklearn.neighbors import KNeighborsClassifier

skf = StratifiedKFold(n_splits=10)
modelKNN = KNeighborsClassifier()

for train_index, test_index in skf.split(data,target):
    X_train, X_val = data.iloc[train_index], data.iloc[test_index]
    y_train, y_val = target.iloc[train_index], target.iloc[test_index]
    
    oversampler = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
        
    min_max = MinMaxScaler().fit(X_train_resampled)
    scaled_X_train = min_max.transform(X_train_resampled)
    scaled_X_val = min_max.transform(X_val)
        
    modelKNN.fit(scaled_X_train, y_train_resampled)
    y_pred = modelKNN.predict(scaled_X_val)
    
    
    print(f1_score(y_val, y_pred))

0.18601446779193936
0.1646643109540636
0.20287868403015763
0.17705821563899415
0.20121951219512196
0.1859518154054971
0.183728813559322
0.200756273633551
0.1829105473965287
0.1902792140641158


In [42]:
from sklearn.tree import DecisionTreeClassifier 

skf = StratifiedKFold(n_splits=10)
modelDT = DecisionTreeClassifier()

for train_index, test_index in skf.split(data,target):
    X_train, X_val = data.iloc[train_index], data.iloc[test_index]
    y_train, y_val = target.iloc[train_index], target.iloc[test_index]
    
    oversampler = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
        
    min_max = MinMaxScaler().fit(X_train_resampled)
    scaled_X_train = min_max.transform(X_train_resampled)
    scaled_X_val = min_max.transform(X_val)
        
    modelDT.fit(scaled_X_train, y_train_resampled)
    y_pred = modelDT.predict(scaled_X_val)
    
    
    print(f1_score(y_val, y_pred ))

0.13642052565707136
0.16196319018404906
0.1526813880126183
0.1488833746898263
0.1561928004881025
0.17169373549883993
0.18
0.18663594470046083
0.18874360432063672
0.21753246753246752


In [ ]:
from sklearn.neural_network import MLPClassifier

skf = StratifiedKFold(n_splits=10)
model = MLPClassifier(random_state=42, max_iter = 400)

for train_index, test_index in skf.split(data,target):
    X_train, X_val = data.iloc[train_index], data.iloc[test_index]
    y_train, y_val = target.iloc[train_index], target.iloc[test_index]
    
    oversampler = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
        
    min_max = MinMaxScaler().fit(X_train_resampled)
    scaled_X_train = min_max.transform(X_train_resampled)
    scaled_X_val = min_max.transform(X_val)
        
    model.fit(scaled_X_train, y_train_resampled)
    y_pred = model.predict(scaled_X_val)
    
    
    print(f1_score(y_val, y_pred ))